In [3]:
#retrieved from https://machinelearningmastery.com/implement-perceptron-algorithm-scratch-python/
#Jason Brownlee

# Perceptron Algorithm on the Sonar Dataset
from random import seed
from random import randrange
from csv import reader
import numpy as np

import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
import numpy as np

print(tf.__version__)

2.19.0


In [5]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

# Make a prediction with weights
def predict(row, weights):
	activation = weights[0]
	for i in range(len(row)-1):
		activation += weights[i + 1] * row[i]
	return 1.0 if activation >= 0.0 else 0.0

# Estimate Perceptron weights using stochastic gradient descent
def train_weights(train, l_rate, n_epoch):
	weights = [0.0 for i in range(len(train[0]))]
	for epoch in range(n_epoch):
		for row in train:
			prediction = predict(row, weights)
			error = row[-1] - prediction
			weights[0] = weights[0] + l_rate * error
			for i in range(len(row)-1):
				weights[i + 1] = weights[i + 1] + l_rate * error * row[i]
	return weights

# Perceptron Algorithm With Stochastic Gradient Descent
def perceptron(train, test, l_rate, n_epoch):
	predictions = list()
	weights = train_weights(train, l_rate, n_epoch)
	for row in test:
		prediction = predict(row, weights)
		predictions.append(prediction)
	return(predictions)

# Test the Perceptron algorithm on the sonar dataset
seed(1)
np.random.seed(1)
# load and prepare data
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
mask_train = np.isin(y_train, [0, 1])
mask_test = np.isin(y_test, [0, 1])
X = np.concatenate([x_train[mask_train], x_test[mask_test]], axis=0)
y = np.concatenate([y_train[mask_train], y_test[mask_test]], axis=0)
n_per_class = 2000
idx0 = np.where(y == 0)[0][:n_per_class]
idx1 = np.where(y == 1)[0][:n_per_class]
idx = np.concatenate([idx0, idx1])
np.random.shuffle(idx)
X = X[idx]
y = y[idx]
X = X.reshape((X.shape[0], -1)).astype('float32') / 255.0
dataset = [list(X[i]) + [float(y[i])] for i in range(len(y))]
# evaluate algorithm
n_folds = 3
l_rate = 0.01
n_epoch = 50
scores = evaluate_algorithm(dataset, perceptron, n_folds, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))


Scores: [98.34958739684922, 98.94973743435858, 98.49962490622656]
Mean Accuracy: 98.600%
